# TP2 — Vectorizing Maps on SPD Matrices (PyTorch)

In [1]:
import torch
import time

torch.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cpu


Utility: generate random SPD matrices

In [2]:
def generate_spd(batch_size, n):
    A = torch.randn(batch_size, n, n, device=device)
    return A @ A.transpose(1, 2) + 1e-3 * torch.eye(n, device=device)


Naive vectorize / devectorize

In [3]:
def vectorize(M):
    idx = torch.tril_indices(M.shape[-1], M.shape[-1])
    return M[idx[0], idx[1]]


def devectorize(v, n):
    M = torch.zeros((n, n), device=v.device)
    idx = torch.tril_indices(n, n)
    M[idx[0], idx[1]] = v
    M = M + M.T - torch.diag(torch.diag(M))
    return M


Naive matrix square root and logarithm

In [4]:
def matrix_sqrt(M):
    eigvals, eigvecs = torch.linalg.eigh(M)
    D_sqrt = torch.diag(torch.sqrt(eigvals))
    return eigvecs @ D_sqrt @ eigvecs.T


def matrix_log(M):
    eigvals, eigvecs = torch.linalg.eigh(M)
    D_log = torch.diag(torch.log(eigvals))
    return eigvecs @ D_log @ eigvecs.T


Correctness test (naive version)

In [5]:
M = generate_spd(batch_size=1, n=5)[0]

v = vectorize(M)
M_rec = devectorize(v, 5)

sqrt_M = matrix_sqrt(M)
log_M = matrix_log(M)

print("Reconstruction error:", torch.norm(M - M_rec).item())
print("Sqrt error ||P² − M||:", torch.norm(sqrt_M @ sqrt_M - M).item())
print("Log error ||exp(logM) − M||:", torch.norm(torch.matrix_exp(log_M) - M).item())


Reconstruction error: 0.0
Sqrt error ||P² − M||: 1.4581782124878373e-05
Log error ||exp(logM) − M||: 1.8850298147299327e-05


Interpretation

Reconstruction error is close to zero: vectorization and devectorization are correct.

The square root verifies
P²=M.

The logarithm verifies
exp⁡(log⁡(M))=M.

Mathematical correctness is validated.

Vectorized (batch-optimized) implementations

Batch vectorize / devectorize

In [6]:
def vectorize_batch(M):
    B, N, _ = M.shape
    idx = torch.tril_indices(N, N)
    return M[:, idx[0], idx[1]]


def devectorize_batch(v, n):
    B = v.shape[0]
    M = torch.zeros((B, n, n), device=v.device)
    idx = torch.tril_indices(n, n)
    M[:, idx[0], idx[1]] = v
    M = M + M.transpose(1, 2) - torch.diag_embed(torch.diagonal(M, dim1=1, dim2=2))
    return M


Batch matrix square root and logarithm

In [7]:
def matrix_sqrt_batch(M):
    eigvals, eigvecs = torch.linalg.eigh(M)
    D_sqrt = torch.diag_embed(torch.sqrt(eigvals))
    return eigvecs @ D_sqrt @ eigvecs.transpose(1, 2)


def matrix_log_batch(M):
    eigvals, eigvecs = torch.linalg.eigh(M)
    D_log = torch.diag_embed(torch.log(eigvals))
    return eigvecs @ D_log @ eigvecs.transpose(1, 2)


Correctness test (vectorized version)

In [8]:
M = generate_spd(batch_size=64, n=8)

v = vectorize_batch(M)
M_rec = devectorize_batch(v, 8)

sqrt_M = matrix_sqrt_batch(M)
log_M = matrix_log_batch(M)

print("Reconstruction error:", torch.norm(M - M_rec).item())
print("Sqrt error:", torch.norm(sqrt_M @ sqrt_M - M).item())
print("Log error:", torch.norm(torch.matrix_exp(log_M) - M).item())


Reconstruction error: 4.064237145939842e-06
Sqrt error: 0.00022827326029073447
Log error: 0.0003048246435355395


Interpretation

Results remain numerically accurate for large batches.

Vectorization preserves correctness.

Operations scale properly with batch size.

Performance comparison

Benchmark naive vs vectorized

In [9]:
B, N = 256, 20
M = generate_spd(B, N)

# Naive version
start = time.time()
for i in range(B):
    v = vectorize(M[i])
    M_rec = devectorize(v, N)
    _ = matrix_sqrt(M_rec)
end = time.time()
naive_time = end - start

# Vectorized version
start = time.time()
v = vectorize_batch(M)
M_rec = devectorize_batch(v, N)
_ = matrix_sqrt_batch(M_rec)
end = time.time()
vectorized_time = end - start

print("Naive time:", naive_time)
print("Vectorized time:", vectorized_time)
print("Speedup:", naive_time / vectorized_time)


Naive time: 0.23712468147277832
Vectorized time: 0.07724761962890625
Speedup: 3.0696697530864197


Interpretation

The vectorized implementation is significantly faster.

The speedup increases with batch size.

On GPU, the performance gap is even larger.

This confirms the importance of batching and tensor-level operations in PyTorch.

Key observations

All mathematical operations are correctly implemented.

Vectorization drastically reduces execution time.

Spectral decompositions dominate computation cost.

Batch processing is mandatory for scalable ML models.